In [1]:
import json
import urllib.request
from penzai import pz

import equinox as eqx
import jax
import jax.numpy as jnp
import torch
from jaxonmodels.vision.resnet import resnet50 as resnet50_jax
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet50


img_name = "doggo.jpeg"
resnet = resnet50(weights=None)
# resnet = resnet18(weights=None)

transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
img = Image.open(img_name)
img_t = transform(img)
print(img_t.shape)  # pyright: ignore
batch_t = torch.unsqueeze(img_t, 0)  # pyright:ignore

# Predict
with torch.no_grad():
    output = resnet(batch_t)
    print(output.shape)  # pyright: ignore
    _, predicted = torch.max(output, 1)

print(
    f"Predicted: {predicted.item()}"
)  # Outputs the ImageNet class index of the prediction

# Load ImageNet labels
# url = "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json"
# with urllib.request.urlopen(url) as url:
#     imagenet_labels = json.loads(url.read().decode())
#
# label = imagenet_labels[str(predicted.item())][1]
# print(f"Label for index {predicted.item()}: {label}")


# jax_resnet, state = resnet18_jax()
jax_resnet, state = resnet50_jax(key=jax.random.PRNGKey(22))
jax_batch = jnp.array(batch_t.numpy())
out, state = eqx.filter_vmap(
    jax_resnet, in_axes=(0, None), out_axes=(0, None), axis_name="batch"
)(jax_batch, state)
print(f"{out.shape}")

# label = imagenet_labels[str(jnp.argmax(out))][1]
# print(f"Label for index {jnp.argmax(out)}: {label}")


PARAMS FOR LAYER1
BLOCK 0 PARAMS:
self.inplanes=64
planes=64
stride=1
downsample=Sequential(
  (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
self.groups=1
self.base_width=64
previous_dilation=1
norm_layer=<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
BLOCK 1 PARAMS:
self.inplanes=256
planes=64
self.groups=1
self.base_width=64
self.dilation=1
BLOCK 2 PARAMS:
self.inplanes=256
planes=64
self.groups=1
self.base_width=64
self.dilation=1
PARAMS FOR LAYER1
BLOCK 0 PARAMS:
self.inplanes=256
planes=128
stride=2
downsample=Sequential(
  (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
self.groups=1
self.base_width=64
previous_dilation=1
norm_layer=<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
BLOCK 1 PARAMS:
self.inplanes=512
planes=128
self.groups=1
self.base_width=64
s

ValueError: conv_general_dilated lhs feature dimension size divided by feature_group_count must equal the rhs input feature dimension size, but 256 // 1 != 64.

In [2]:
from penzai import pz 

with pz.ts.active_autovisualizer.set_scoped(pz.ts.ArrayAutovisualizer()):
  pz.ts.display(jax_resnet)

In [3]:
import jax.numpy as jnp

sd = resnet.state_dict()
resnet_state_dict = {
    k: jnp.array(sd[k].numpy()) for k in sd if "weight" in k
}

with pz.ts.active_autovisualizer.set_scoped(pz.ts.ArrayAutovisualizer()):
  pz.ts.display(resnet_state_dict)